In [ ]:
import pandas as pd
import numpy as np
import re

import itertools
import requests
from wordcloud import WordCloud

from rich import inspect
from rich.pretty import pprint
from tqdm.notebook import tqdm

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import matplotlib.pyplot as plt

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [ ]:
recipes_path = "../data/full_dataset.csv"

recipes_data = pd.read_csv(
    recipes_path,
    index_col=0,
    usecols=["title", "link", "NER"],
    dtype={
        "title": "string[pyarrow]",
        "link": "string[pyarrow]",
        "NER": "string[pyarrow]",
    },
)

print(f" data shape: {recipes_data.shape}")
print(recipes_data.memory_usage(deep=True))

In [ ]:
recipes_data.sample()

In [ ]:
recipes_data["website"] = recipes_data.link.str.split("/").str.get(0)

recipes_data.sample(5)

In [ ]:
recipes_data.website.value_counts()

After inspection of some websites, the following list of websites will be re-scraped for further features (including duration, nutrition facts and servings):

- www.food.com
- www.allrecipes.com
- www.tasteofhome.com

In [ ]:
preliminary_websites = [
    "www.food.com",
    "www.allrecipes.com",
    "www.tasteofhome.com",
]

filtered_data = recipes_data.query(
    f" website == {preliminary_websites} "
).copy()

filtered_data.NER = filtered_data.NER.apply(eval)

filtered_data.shape

In [ ]:
all_ingredients = filtered_data.NER.explode()

ingredients_frequency = all_ingredients.value_counts().to_dict()

In [ ]:
wordcloud = WordCloud(width=1000, height=500).generate_from_frequencies(
    ingredients_frequency
)
plt.figure(figsize=(15, 8))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
filtered_data.isnull().sum()

In [ ]:
filtered_data.NER.iloc[0]

In [ ]:
filtered_data["duration"] = np.nan
filtered_data["nutrition_facts"] = np.nan
filtered_data["servings"] = np.nan

In [ ]:
filtered_data.to_csv("ommak.csv", index=False)

# Start of scraping:

## www.food.com

In [13]:
import pandas as pd
import numpy as np
import re
import time
import itertools
import requests
from bs4 import BeautifulSoup
from wordcloud import WordCloud

from rich import inspect
from tqdm.notebook import tqdm
from pprint import pprint

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.common.action_chains import ActionChains

from selenium_stealth import stealth


import matplotlib.pyplot as plt

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [5]:
filtered_data = pd.read_csv(
    "/Users/man-top/my_github/FoodFlex/notebooks/ommak.csv"
)

example_to_scrape = (
    filtered_data.query(" website == 'www.allrecipes.com' ")
    .link.sample()
    .iloc[0]
)

example_to_scrape

'www.allrecipes.com/recipe/17096/blue-cheese-dip-i/'

In [36]:
import time

chrome_driver_path = (
    "/Users/man-top/Downloads/chromedriver_mac_arm64/chromedriver"
)


service = Service(chrome_driver_path)
options = webdriver.ChromeOptions()
options.headless = False

driver = webdriver.Chrome(service=service, options=options, keep_alive=True)

driver.get(f"https://{example_to_scrape}")

time.sleep(4)

# scrolling
body = driver.find_element(by=By.TAG_NAME("body"))
body.send_keys(Keys.PAGE_DOWN)

time.sleep(2)


button_url = WebDriverWait(driver, 30).until(
    EC.element_to_be_clickable(
        (By.CSS_SELECTOR, "button.link.facts__nutrition.svelte-ovaflp")
    )
)
# button_url = WebDriverWait(driver, 20).until(lambda x: x.find_element(By.XPATH, "//*[@id='recipe']/div[9]/button"))

# inspect(button_url)

try:
    button_url.click()
    print("yel3an rabbak")
except WebDriverException:
    print("Element is not clickable")


time.sleep(4)

try:
    nutrition_facts_url = WebDriverWait(driver, 30).until(
        EC.visibility_of_element_located(
            (By.XPATH, "//*[@id='top']/div[3]/div[4]/div/div/div[2]")
        )
    )
except:
    print("not found")
print()

driver.quit()

/var/folders/mg/yzrszbkn6rs6h28ffl15nmyr0000gn/T/ipykernel_1165/2840394872.py:7: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = False


TypeError: 'str' object is not callable

In [ ]:
inspect(button_url)

In [7]:
def setup_web_driver(headless: bool):
    chrome_driver_path = (
        "/Users/man-top/Downloads/chromedriver_mac_arm64/chromedriver"
    )

    service = Service(chrome_driver_path)
    options = webdriver.ChromeOptions()
    if headless:
        options.add_argument("--headless=new")

    driver = webdriver.Chrome(service=service, options=options, keep_alive=True)
    return driver

In [29]:
def scrape_nutrition_facts(driver, url):
    driver.get(f"https://{url}")

    time.sleep(np.random.randint(2, 4))

    button_url = driver.find_element(
        By.XPATH, "//button[contains(text(), 'Nutrition information')]"
    )

    time.sleep(np.random.randint(2, 5))

    driver.execute_script("arguments[0].scrollIntoView();", button_url)

    expanded = button_url.get_attribute("aria-expanded")

    print("start: ", expanded)

    count_failed_click = 0

    while expanded == "false":
        print(expanded)
        if count_failed_click % 4 == 0:
            print("failed 4 times in a row... Refreshing")
            time.sleep(np.random.randint(1, 2))
            driver.refresh()

        time.sleep(np.random.randint(1, 2))

        button_url = driver.find_element(
            By.XPATH, "//button[contains(text(), 'Nutrition information')]"
        )

        time.sleep(np.random.randint(2, 5))

        button_url.click()
        expanded = button_url.get_attribute("aria-expanded")

        count_failed_click += 1

    time.sleep(1)

    nutrition_facts_url = driver.find_element(
        By.CSS_SELECTOR, "div.recipe-nutrition.svelte-epeb0m"
    )
    driver.close()
    driver.quit()
    return nutrition_facts_url.text

In [6]:
import time

chrome_driver_path = (
    "/Users/man-top/Downloads/chromedriver_mac_arm64/chromedriver"
)

service = Service(chrome_driver_path)
options = webdriver.ChromeOptions()

options.add_argument("start-maximized")


# options.add_argument('--headless=new')

driver = webdriver.Chrome(service=service, options=options, keep_alive=True)


driver.get(f"https://{example_to_scrape}")

time.sleep(np.random.randint(2, 5))

driver.fullscreen_window()

time.sleep(np.random.randint(2, 5))

button_url = driver.find_element(
    By.XPATH, "//*[@id='mntl-nutrition-facts-label_1-0']/button/span[1]"
)

time.sleep(np.random.randint(2, 5))

driver.execute_script("arguments[0].scrollIntoView();", button_url)

# expanded = button_url.get_attribute('aria-expanded')

# print(expanded)

count_failed_click = 0

while expanded == "false":
    print(expanded)
    if count_failed_click % 4 == 0:
        print("failed 4 times in a row... Refreshing")
        time.sleep(np.random.randint(1, 2))
        driver.refresh()

    time.sleep(np.random.randint(1, 2))

    button_url = driver.find_element(
        By.XPATH, "//button[contains(text(), 'Nutrition information')]"
    )

    time.sleep(np.random.randint(2, 5))

    button_url.click()
    expanded = button_url.get_attribute("aria-expanded")

    count_failed_click += 1


time.sleep(1)

nutrition_facts_url = driver.find_element(
    By.CSS_SELECTOR, "div.recipe-nutrition.svelte-epeb0m"
)

print(nutrition_facts_url.text)


driver.close()
driver.quit()

false
false
failed 4 times in a row... Refreshing


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=114.0.5735.198)
Stacktrace:
0   chromedriver                        0x0000000102e83f48 chromedriver + 4226888
1   chromedriver                        0x0000000102e7c4f4 chromedriver + 4195572
2   chromedriver                        0x0000000102ac0d68 chromedriver + 281960
3   chromedriver                        0x0000000102a9aef8 chromedriver + 126712
4   chromedriver                        0x0000000102b1fe88 chromedriver + 671368
5   chromedriver                        0x0000000102b3245c chromedriver + 746588
6   chromedriver                        0x0000000102aeff1c chromedriver + 474908
7   chromedriver                        0x0000000102af0ef4 chromedriver + 478964
8   chromedriver                        0x0000000102e4559c chromedriver + 3970460
9   chromedriver                        0x0000000102e496f0 chromedriver + 3987184
10  chromedriver  

In [8]:
driver = setup_web_driver(headless=False)

scrape_nutrition_facts(driver=driver, url=example_to_scrape)

NameError: name 'scrape_nutrition_facts' is not defined

In [4]:
def scrape_all_recipes_nutrition_facts(url):
    print(f"\n {url}")
    driver = setup_web_driver(headless=True)

    driver.get(f"https://{url}")

    time.sleep(np.random.randint(1, 3))

    try:
        driver.find_element(By.ID, "onetrust-reject-all-handler").click()
    except:
        print("nop reject all detected!")

    time.sleep(np.random.randint(2, 5))

    try:
        button_url = driver.find_element(
            By.XPATH, "//*[@id='mntl-nutrition-facts-label_1-0']/button/span[1]"
        )

        time.sleep(np.random.randint(2, 5))

        driver.execute_script("arguments[0].scrollIntoView();", button_url)

        driver.execute_script("window.scrollBy(0,-100)", "")

        time.sleep(np.random.randint(1, 3))

        button_url.click()

        time.sleep(np.random.randint(2, 4))

        nutrition_facts_url = driver.find_element(
            By.XPATH, "//*[@id='mntl-nutrition-facts-label_1-0']/div"
        )

        return nutrition_facts_url.text
    except:
        print("this recipe is doomed!")

    driver.close()
    driver.quit()
    return np.nan

In [45]:
import time

chrome_driver_path = (
    "/Users/man-top/Downloads/chromedriver_mac_arm64/chromedriver"
)

service = Service(chrome_driver_path)
options = webdriver.ChromeOptions()

# options.add_argument('--headless=new')

driver = webdriver.Chrome(service=service, options=options, keep_alive=True)


driver.get(f"https://{example_to_scrape}")

time.sleep(np.random.randint(1, 3))


reject_all = driver.find_element(By.ID, "onetrust-reject-all-handler")
reject_all.click()

time.sleep(np.random.randint(2, 5))

button_url = driver.find_element(
    By.XPATH, "//*[@id='mntl-nutrition-facts-label_1-0']/button/span[1]"
)

time.sleep(np.random.randint(2, 5))

driver.execute_script("arguments[0].scrollIntoView();", button_url)

driver.execute_script("window.scrollBy(0,-100)", "")

time.sleep(np.random.randint(1, 3))

button_url.click()

time.sleep(np.random.randint(2, 4))

nutrition_facts_url = driver.find_element(
    By.XPATH, "//*[@id='mntl-nutrition-facts-label_1-0']/div"
)

print(nutrition_facts_url.text)

driver.close()
driver.quit()

Nutrition Facts
Servings Per Recipe 6
Calories 492
% Daily Value *
Total Fat 39g 50%
Saturated Fat 15g 74%
Cholesterol 224mg 75%
Sodium 123mg 5%
Total Carbohydrate 1g 0%
Dietary Fiber 0g 1%
Total Sugars 0g
Protein 32g
Vitamin C 3mg 16%
Calcium 14mg 1%
Iron 4mg 24%
Potassium 323mg 7%
* Percent Daily Values are based on a 2,000 calorie diet. Your daily values may be higher or lower depending on your calorie needs.
** Nutrient information is not available for all ingredients. Amount is based on available nutrient data.
(-) Information is not currently available for this nutrient. If you are following a medically restrictive diet, please consult your doctor or registered dietitian before preparing this recipe for personal consumption.
Powered by the ESHA Research Database © 2018, ESHA Research, Inc. All Rights Reserved


In [71]:
scrape_all_recipes_nutrition_facts(
    url="www.allrecipes.com/recipe/20770/quite-a-peachy-pour/"
)


 www.allrecipes.com/recipe/20770/quite-a-peachy-pour/
this recipe is doomed!


nan

In [9]:
all_recipes_source_df = filtered_data.query(" website == 'www.allrecipes.com' ")

nutrition_facts = []
for link in all_recipes_source_df.link.values:
    nutrition_facts.append(scrape_all_recipes_nutrition_facts(url=link))


 www.allrecipes.com/recipe/241895/deconstructed-screwdriver-the-raw-egg/


NameError: name 'time' is not defined

In [66]:
all_recipes_source_df.shape

(61398, 6)

In [76]:
len(nutrition_facts)

3779

In [78]:
nutrition_facts_series = pd.Series(
    data=nutrition_facts,
    index=all_recipes_source_df.index[: len(nutrition_facts)],
)
nutrition_facts_series.name = "Nutrition_facts_unstructured"
all_recipes_source_df = all_recipes_source_df.join(nutrition_facts_series)
all_recipes_source_df.to_csv("all_recipes_data_w_nutrition_facts.csv")

In [79]:
all_recipes_source_df.head()

,link,NER,website,duration,nutrition_facts,servings,Nutrition_facts_unstructured
0,www.allrecipes.com/recipe/241895/deconstructed...,"['orange juice', 'jiggers vodka']",www.allrecipes.com,NaN,NaN,NaN,Nutrition Facts\nServings Per Recipe 1\nCalori...
1,www.allrecipes.com/recipe/20808/kettle-corn/,"['vegetable oil', 'white sugar', 'popcorn kern...",www.allrecipes.com,NaN,NaN,NaN,Nutrition Facts\nServings Per Recipe 5\nCalori...
2,www.allrecipes.com/recipe/238733/pops-fabulous...,"['eggplants', 'green bell peppers', 'red bell ...",www.allrecipes.com,NaN,NaN,NaN,Nutrition Facts\nServings Per Recipe 10\nCalor...
3,www.allrecipes.com/recipe/241254/chipotle-mang...,"['avocados', 'tomatoes', 'mango', 'cilantro', ...",www.allrecipes.com,NaN,NaN,NaN,Nutrition Facts\nServings Per Recipe 20\nCalor...
4,www.allrecipes.com/recipe/261696/basic-homemad...,"['milk', 'heavy whipping cream', 'white vinega...",www.allrecipes.com,NaN,NaN,NaN,Nutrition Facts\nServings Per Recipe 4\nCalori...


In [81]:
all_recipes_source_df.Nutrition_facts_unstructured.isnull().sum() / len(
    all_recipes_source_df
)

0.9400794814163328

In [82]:
len(nutrition_facts_series) / len(all_recipes_source_df)

0.06154923613147008

In [84]:
len(all_recipes_source_df), len(nutrition_facts_series)

(61398, 3779)

# continuation of scraping:

In [12]:
import pandas as pd
import numpy as np
import re
import time
import itertools
import requests
from bs4 import BeautifulSoup
from wordcloud import WordCloud

from rich import inspect
from tqdm import tqdm
from pprint import pprint

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.common.action_chains import ActionChains

from selenium_stealth import stealth


import matplotlib.pyplot as plt

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [13]:
def scrape_all_recipes_nutrition_facts(url):
    print(f"\n {url}")
    driver = setup_web_driver(headless=True)

    driver.get(f"https://{url}")

    time.sleep(np.random.randint(1, 3))

    try:
        driver.find_element(By.ID, "onetrust-reject-all-handler").click()
    except:
        print("nop reject all detected!")

    time.sleep(np.random.randint(2, 3))

    try:
        button_url = driver.find_element(
            By.XPATH, "//*[@id='mntl-nutrition-facts-label_1-0']/button/span[1]"
        )

        time.sleep(np.random.randint(2, 3))

        driver.execute_script("arguments[0].scrollIntoView();", button_url)

        driver.execute_script("window.scrollBy(0,-100)", "")

        time.sleep(np.random.randint(1, 2))

        button_url.click()

        time.sleep(np.random.randint(1, 2))

        nutrition_facts_url = driver.find_element(
            By.XPATH, "//*[@id='mntl-nutrition-facts-label_1-0']/div"
        )

        return nutrition_facts_url.text
    except:
        print("this recipe is doomed!")

    driver.close()
    driver.quit()
    return np.nan

def setup_web_driver(headless: bool):
    chrome_driver_path = (
        "/Users/man-top/Downloads/chromedriver_mac_arm64/chromedriver"
    )

    service = Service(chrome_driver_path)
    options = webdriver.ChromeOptions()
    if headless:
        options.add_argument("--headless=new")

    driver = webdriver.Chrome(service=service, options=options, keep_alive=True)
    return driver

In [17]:

all_recipes_source_df = pd.read_csv("all_recipes_data_w_nutrition_facts.csv")
try:
    for link in tqdm(all_recipes_source_df.iloc[4543:].query(
        " Nutrition_facts_unstructured.isnull() "
    ).link.values):
        all_recipes_source_df.loc[
            all_recipes_source_df.query(f" link == '{link}' ").index,
            "Nutrition_facts_unstructured",
        ] = scrape_all_recipes_nutrition_facts(url=link)
except (KeyboardInterrupt, WebDriverException):
    all_recipes_source_df.to_csv("all_recipes_data_w_nutrition_facts.csv", index=False)


  0%|          | 0/56854 [00:00<?, ?it/s]


 www.allrecipes.com/recipe/230987/crisp-apples-with-citrus-dressing/


  0%|          | 1/56854 [00:10<161:45:58, 10.24s/it]


 www.allrecipes.com/recipe/16016/lentil-soup/


  0%|          | 2/56854 [00:15<111:49:07,  7.08s/it]

...


 www.allrecipes.com/recipe/255694/meatless-buffalo-dip/


  0%|          | 86/56854 [13:19<142:27:44,  9.03s/it]


 www.allrecipes.com/recipe/219922/wendys-zucchini-bread/


  0%|          | 87/56854 [13:29<143:20:09,  9.09s/it]


 www.allrecipes.com/recipe/237622/banana-crush/


In [ ]:
all_recipes_source_df.query("link=='www.allrecipes.com/recipe/255995/gourmet-chocolate-covered-cherry-jell-o-shots/'")

,Unnamed: 0,link,NER,website,duration,nutrition_facts,servings,Nutrition_facts_unstructured
4027,4027,www.allrecipes.com/recipe/255995/gourmet-choco...,"['gelatin', 'boiling water', 'vodka', 'chocola...",www.allrecipes.com,NaN,NaN,NaN,Nutrition Facts\nServings Per Recipe 12\nCalor...


In [16]:
all_recipes_source_df.query("Nutrition_facts_unstructured.notnull()").index[-1]

4543